In [13]:
from threading import Thread
import socket
import json
from multiprocessing import Pool,Process
import pandas as pd
import numpy as np
import collections
import re
import nltk
from sklearn.decomposition import PCA


In [ ]:
English_f=pd.read_csv('all_news.csv')


stoplist = ['very', 'ourselves', 'am', 'doesn', 'through', 'me', 'against', 'up', 'just', 'her', 'ours', 
            'couldn', 'because', 'is', 'isn', 'it', 'only', 'in', 'such', 'too', 'mustn', 'under', 'their', 
            'if', 'to', 'my', 'himself', 'after', 'why', 'while', 'can', 'each', 'itself', 'his', 'all', 'once', 
            'herself', 'more', 'our', 'they', 'hasn', 'on', 'ma', 'them', 'its', 'where', 'did', 'll', 'you', 
            'didn', 'nor', 'as', 'now', 'before', 'those', 'yours', 'from', 'who', 'was', 'm', 'been', 'will', 
            'into', 'same', 'how', 'some', 'of', 'out', 'with', 's', 'being', 't', 'mightn', 'she', 'again', 'be', 
            'by', 'shan', 'have', 'yourselves', 'needn', 'and', 'are', 'o', 'these', 'further', 'most', 'yourself', 
            'having', 'aren', 'here', 'he', 'were', 'but', 'this', 'myself', 'own', 'we', 'so', 'i', 'does', 'both', 
            'when', 'between', 'd', 'had', 'the', 'y', 'has', 'down', 'off', 'than', 'haven', 'whom', 'wouldn', 
            'should', 've', 'over', 'themselves', 'few', 'then', 'hadn', 'what', 'until', 'won', 'no', 'about', 
            'any', 'that', 'for', 'shouldn', 'don', 'do', 'there', 'doing', 'an', 'or', 'ain', 'hers', 'wasn', 
            'weren', 'above', 'a', 'at', 'your', 'theirs', 'below', 'other', 'not', 're', 'him', 'during', 'which']

#分词
lemmatizer=nltk.stem.WordNetLemmatizer()
def wordcount(news):
    freq=[]
    news=re.sub(r'\W+|\d+',' ',news)
    for word in news.split():
        if word not in stoplist:
            freq.append(lemmatizer.lemmatize(word))
    return ' '.join(freq)
English_f['word']=English_f['body'].apply(wordcount)
words=English_f['word'].str.split(' ',expand=True).stack().rename('words').reset_index()
words['level_0']=words['level_0']+1
words.columns=['id','level_1','words']
print(words)

#计算TF
def counttf(x):
    l=len(x)
    p=x['words'].value_counts()
    d={'words':p.index,
       'TF':p.values/l}
    p_2d=pd.DataFrame(data=d)

    r=pd.merge(x,p_2d,on='words')
    return r

TF_data=words.groupby('id').apply(counttf)
#统计完频率之后 去除重复词
TF_data.drop_duplicates(subset=['id','words'],inplace=True)

#计算IDF
doc_num=len(English_f)
def countidf(x):
    df_t=len(x['id'].value_counts())
    return np.log(doc_num/df_t)

IDF_data=words.groupby('words').apply(countidf).reset_index()
IDF_data.columns=['words','IDF']
tf_idf=pd.merge(TF_data,IDF_data)
tf_idf['TF-IDF']=tf_idf['TF']*tf_idf['IDF']
tf_idf.to_csv('tfidf.csv')


          id  level_1      words
0          1        0  Quarterly
1          1        1     profit
2          1        2         US
3          1        3     medium
4          1        4      giant
...      ...      ...        ...
507047  2225     1636     Online
507048  2225     1637       game
507049  2225     1638      ahhhh
507050  2225     1639        day
507051  2225     1640        LOL

[507052 rows x 3 columns]


In [ ]:
#转化为词向量
print(tf_idf['words'])
a=list(set(tf_idf['words']))
k=len(a)
word_list=dict(zip(a,list(range(k))))
wordlst=pd.Series(word_list)

def wordtovec(x):
    vec=np.zeros(k,)
    for i in range(x.shape[0]):
        idx=word_list[x.iloc[i]['words']]
        vec[idx]=x.iloc[i]['TF-IDF']
    return vec
newdata=tf_idf.groupby('id').apply(wordtovec)
print(newdata)

#降维
new_data=np.vstack(newdata.values)

model=PCA(n_components=300)
dev=model.fit_transform(new_data)

print(dev)

In [ ]:
#计算各个文章之间的相似度
for i in range(dev.shape[0]):
    dev[i]/=np.linalg.norm(dev[i])
cos_sim=dev@dev.T

print(cos_sim.shape)
cos_sim=cos_sim-np.eye(2225)
print(cos_sim)

In [ ]:
#利用tf-idf矩阵，计算词之间的相似度
new_data=np.load('tf_idf.npy')
matrix=new_data.T@new_data

x=matrix.shape[0]
idxar=np.argsort(matrix,axis=1)[:,-4:-1]#取相似度排序前三个为模糊词
synonym=word_list
for word in word_list:
    idxlst=idxar[word_list[word]]
    lst=[]
    for k in idxlst:
        lst.append(a[k])#a是前面提到的词典，它和word_list互为补充，使查询时间缩短到O(1)
    synonym[word]=lst

In [ ]:
#将之前各步结果保存下来，ndarray用.npy保存，dict用json保存
np.save('tf_idf.npy',new_data)
np.save('cos_sim.npy',cos_sim)
np.save('dev.npy',dev)
f=open('wordlist.txt','w')
f.write(json.dumps(word_list))
f.close()
f=open('synonym.txt','w')#将模糊词典保存下来
f.write(json.dumps(synonym))
f.close()

###以下为测试部分，其中各步运算结果已经保存为文件，因此加载比较快

In [1]:
from threading import Thread
import socket
import json
from multiprocessing import Pool,Process
import pandas as pd
import numpy as np
import collections
import re
import nltk
from sklearn.decomposition import PCA
English_f=pd.read_csv('all_news.csv')
cos_sim=np.load('cos_sim.npy')
with open('wordlist.txt') as f:#空间换时间的哈希表
    word_list=json.load(f)
tf_idf=pd.read_csv('tfidf.csv')
topics=np.array(English_f['topic'])
titles=np.array(English_f['title'])
with open('synonym.txt') as f:#模糊词检索表
    synonym=json.load(f)

In [2]:
class LocalServer(object):
    def __init__(self, host, port):
        self.address = (host, port)
    #这一个函数依据输入的word列表，输出所有匹配的articles  
    #传参包括connection和word列表，最后直接在此函数中将结果返回，做到了并发处理
    def lookupfor(self,words,conn):
        lemmatizer=nltk.stem.WordNetLemmatizer()
        for i in range(len(words)):
            words[i]=lemmatizer.lemmatize(words[i])#使用lemmatizer进行词性还原
            words+=synonym[words[i]]#将模糊词添加到列表中
        global topics
        global titles
        k=len(words)
        l=len(topics)
        returnlist=[]
        #首先检索主题和题目，这里认为二者重要性相同，如果又出现了一次，认为其重要性更高，排在前面
        for j in range(k):
            for i in range(l):
                if topics[i]==words[j]:
                    if i in returnlist:
                        returnlist[returnlist.index[i]],returnlist[0]=returnlist[0],returnlist[returnlist.index[i]]
                    else:
                        returnlist.append(i)
                if titles[i]==words[j]:
                    if i in returnlist:
                        returnlist[returnlist.index[i]],returnlist[0]=returnlist[0],returnlist[returnlist.index[i]]
                    else:
                        returnlist.append(i)
        #其次检索与已有文章相似度最高的一篇文章，依次放入returnlist
        for num in returnlist:
            idx=np.argmax(cos_sim[num])
            if idx in returnlist:
                returnlist[returnlist.index[idx]],returnlist[0]=returnlist[0],returnlist[returnlist.index[idx]]
            else:
                returnlist.append(idx)
        #最后按关键字进行查询，按TF-IDF由高到低依次输出
        for j in range(k):
            if words[j] in word_list:
                df=tf_idf[tf_idf['words']==words[j]]
                for i in df['id']:
                    if i not in returnlist:
                        returnlist.append(i-1)
        articles=[]
        for k in returnlist:
            articles.append(tuple(English_f.iloc[k,0:2]))

        conn.send(json.dumps(articles).encode('utf-8'))#将结果转化为json字符串后返回客户端
        return articles
    
    def run(self):
        server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        server.bind(self.address)
        i=0
        server.listen(5)
        while True:
            conn,add=server.accept()
            print(f'Connected to {add}')
            
            words=json.loads(conn.recv(1024).decode('utf-8'))#接收传参
            if 'break_000' in words:                        #此处认为break_000为结束符
                break
            
            print(words)    
            p=Thread(target=self.lookupfor,args=(words,conn))#多线程实现并发处理
            p.start()
        p.join()
        server.close()
    
            
            
        
     
        

#### 运行服务器端
启动服务器之后，在run.ipynb中运行客户端图形界面

In [ ]:
server = LocalServer("127.0.0.1", 1234)
server.run()

Connected to ('127.0.0.1', 62499)
['science']
